In [1]:
import sys

import random 

from src.tdde13.data_handling import * #
from src.tdde13.evaluation import * 
from sklearn.feature_extraction.text import CountVectorizer # we could have use TfidfVectorizer too. From sklearn doc : "tf-idf vectors are also known to work well in practice"
from sklearn.preprocessing import LabelEncoder

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
X, y, edge_index, idx_train, idx_val, idx_test = get_data_graphsage()

10000
9892


In [3]:
n_nodes = X.shape[0]
edge_index = torch.tensor([[i,i] for i in range(n_nodes)])


In [4]:
edge_index.shape

torch.Size([9892, 2])

In [6]:
use_glove = True
if use_glove:
    X_transformed = glove_embedding(X)
else:
    vectorizer = CountVectorizer()
    X_transformed = vectorizer.fit_transform(X)
    X_transformed = construct_sparse_tensor(X_transformed.tocoo())

In [9]:
from src.tdde13.graphsage import GraphSAGE

y = LabelEncoder().fit_transform(y)
y = torch.tensor(y)   

# use_glove = False

in_dim = X_transformed.shape[1] 
hidden_dim = 128
out_dim = 10

'''
    cv
    "lr" : 0.001,
    "epochs" : 35, # 15,
    "batch_size" : 128,
    "patience" : 5, # 5
'''

hparams = {
    "lr" : 0.001,
    "epochs" : 35, # 15,
    "batch_size" : 128,
    "patience" : 5, # 5
    "use_glove" : use_glove
}

graphsage = GraphSAGE(in_dim=in_dim, hidden_dim=hidden_dim, out_dim=out_dim)

# graphsage.train_graphsage(X_transformed, edge_index, y, idx_train, idx_val, idx_test)
trace_train, trace_val = graphsage.train_graphsage_batchwise(X_transformed, edge_index, y, idx_train, idx_val, idx_test, hparams)

plot_curves(trace_train, trace_val)

Create Data Object
Epoch: 1


100%|██████████| 8/8 [00:00<00:00, 14.64it/s]


train loss: 281.5767 train acc: 0.1245 val loss: 19.5093 val acc: 0.1010
Epoch: 2


 11%|█▏        | 7/62 [00:05<00:42,  1.28it/s]